In [1]:
import argparse
import copy
import csv
import os
import warnings
import numpy
import torch
import tqdm
import yaml
from torch.utils import data
from nets import nn
from utils import util
from utils.dataset import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
tb_writer = SummaryWriter() #create the log_file 
import cv2

In [2]:
@torch.no_grad()
def test(args, params, model=None):
    filenames = []
    
    with open('../data/test.txt') as reader:
        for filename in reader.readlines():
            filename = filename.rstrip().split('/')[-1]
            filenames.append('../data/new_data/test/' + filename)

    dataset = Dataset(filenames, 640, params, False)
    loader = data.DataLoader(dataset, 8, False, num_workers=8,
                             pin_memory=True, collate_fn=Dataset.collate_fn)

    if model is None:
        model = torch.load('./weights/best.pt', map_location='cuda')['model'].float()

    model.half()
    model.eval()
    # Configure
    iou_v = torch.linspace(0.5, 0.95, 10).cuda()  # iou vector for mAP@0.5:0.95
    n_iou = iou_v.numel()

    m_pre = 0.
    m_rec = 0.
    map50 = 0.
    mean_ap = 0.
    metrics = []
    p_bar = tqdm.tqdm(loader, desc=('%10s' * 3) % ('precision', 'recall', 'mAP'))
    for samples, targets, shapes in p_bar:
        samples = samples.cuda()
        targets = targets.cuda()
        samples = samples.half()  # uint8 to fp16/32
        samples = samples / 255  # 0 - 255 to 0.0 - 1.0
        _, _, height, width = samples.shape  # batch size, channels, height, width

        # Inference
        outputs = model(samples)
        # NMS
        targets[:, 2:] *= torch.tensor((width, height, width, height)).cuda()  # to pixels
        outputs = util.non_max_suppression(outputs, 0.001, 0.65)

        # Metrics
        for i, output in enumerate(outputs):
            labels = targets[targets[:, 0] == i, 1:]
            correct = torch.zeros(output.shape[0], n_iou, dtype=torch.bool).cuda()

            if output.shape[0] == 0:
                if labels.shape[0]:
                    metrics.append((correct, *torch.zeros((3, 0)).cuda()))
                continue

            detections = output.clone()
            util.scale(detections[:, :4], samples[i].shape[1:], shapes[i][0], shapes[i][1])

            # Evaluate
            if labels.shape[0]:
                tbox = labels[:, 1:5].clone()  # target boxes
                tbox[:, 0] = labels[:, 1] - labels[:, 3] / 2  # top left x
                tbox[:, 1] = labels[:, 2] - labels[:, 4] / 2  # top left y
                tbox[:, 2] = labels[:, 1] + labels[:, 3] / 2  # bottom right x
                tbox[:, 3] = labels[:, 2] + labels[:, 4] / 2  # bottom right y
                util.scale(tbox, samples[i].shape[1:], shapes[i][0], shapes[i][1])

                correct = numpy.zeros((detections.shape[0], iou_v.shape[0]))
                correct = correct.astype(bool)

                t_tensor = torch.cat((labels[:, 0:1], tbox), 1)
                iou = util.box_iou(t_tensor[:, 1:], detections[:, :4])
                correct_class = t_tensor[:, 0:1] == detections[:, 5]
                for j in range(len(iou_v)):
                    x = torch.where((iou >= iou_v[j]) & correct_class)
                    if x[0].shape[0]:
                        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1)
                        matches = matches.cpu().numpy()
                        if x[0].shape[0] > 1:
                            matches = matches[matches[:, 2].argsort()[::-1]]
                            matches = matches[numpy.unique(matches[:, 1], return_index=True)[1]]
                            matches = matches[numpy.unique(matches[:, 0], return_index=True)[1]]
                        correct[matches[:, 1].astype(int), j] = True
                correct = torch.tensor(correct, dtype=torch.bool, device=iou_v.device)
            metrics.append((correct, output[:, 4], output[:, 5], labels[:, 0]))

    # Compute metrics
    metrics = [torch.cat(x, 0).cpu().numpy() for x in zip(*metrics)]  # to numpy
    if len(metrics) and metrics[0].any():
        tp, fp, m_pre, m_rec, map50, mean_ap = util.compute_ap(*metrics)

    # Print results
    print('%10.3g' * 3 % (m_pre, m_rec, mean_ap))

    # Return results
    model.float()  # for training
    return map50, mean_ap, m_pre, m_rec

In [3]:
args = {
    'input_size': 640,
    'batch_size': 16,
    'local_rank': 0,  # This might be irrelevant in a non-distributed setup
    'epochs': 1,
    'train': True,  # Set to False if you don't want to train
    'test': False,  # Set to True if you want to test
    'world_size': 1 # Assuming a single-process setup
}

# Adjust for potential distributed computing environments, even though it might not be applicable
args['local_rank'] = int(os.getenv('LOCAL_RANK', 0))
args['world_size'] = int(os.getenv('WORLD_SIZE', 1))

if args['world_size'] > 1:
    torch.cuda.set_device(device=args['local_rank'])
    torch.distributed.init_process_group(backend='nccl', init_method='env://')

if args['local_rank'] == 0:
    if not os.path.exists('weights'):
        os.makedirs('weights')

# Assuming util is a module with these functions. If not, you'll need to define them or adjust accordingly.
util.setup_seed()
util.setup_multi_processes()

# Load parameters from args.yaml
with open('./utils/args_indoor_openImages.yaml', 'r') as f:
    params = yaml.safe_load(f)
args_namespace = argparse.Namespace(**args)

In [7]:
model = torch.load('./weights/BESTLastlayerTransformerwithDarkfpn320epochs.pt', map_location='cuda')['model'].float()
ema = util.EMA(model)

last = test(args, params, ema.ema)
tb_writer.add_scalar('mAP', last[1], global_step=epoch)
tb_writer.add_scalar('mAP@50', last[0], global_step=epoch)
tb_writer.add_scalar('Precision', last[2], global_step=epoch)
tb_writer.add_scalar('recall', last[3], global_step=epoch)
writer.writerow({'mAP': str(f'{last[1]:.3f}'),
                 'epoch': str(epoch + 1).zfill(3),
                 'mAP@50': str(f'{last[0]:.3f}')})

 precision    recall       mAP:   0%|                                                         | 0/2067 [00:00<?, ?it/s]C:\Users\SalahEddine.Laidoudi\miniconda3\envs\ultra\lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
 precision    recall       mAP: 100%|██████████████████████████████████████████████| 2067/2067 [05:56<00:00,  5.79it/s]


     0.625     0.516     0.384


NameError: name 'epoch' is not defined